In [1]:
'''
Created on 2018. 2. 14.

@author: phs
'''
"""
1.대화 말뭉치 파일을 읽어들인다.
2.대화 말뭉치를 읽어서 자연어 처리 및  Bag of word 생성
3.Bag of word를 딥러닝 알고리즘 활용을 위한 입력으로 변환
4.딥러닝(tensorflow)을 통한 자연어 이해 모델 생성
5.자연어 이해 모델을 관리한다(저장,읽기)
"""
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# things we need for NLP
import json
import pickle
import random

# import nltk
# from nltk.stem.lancaster import LancasterStemmer
# stemmer = LancasterStemmer()

# from konlpy.tag import Komoran
from konlpy.tag import Okt
# komoran = Komoran()
twitter = Okt()

# import tflearn
import tensorflow as tf
import numpy as np



In [2]:
tf.__version__

'2.3.0'

In [3]:
def read_dialog_intents_jsonfile(input_file_name):
    """
     대화 말뭉치 파일을 읽어들인다.
    """
    with open(input_file_name, 'rt', encoding='UTF8') as json_data:
        intents = json.load(json_data)
        
    return intents
    


In [4]:
def dialog_nlp_processing(intents):
    """
     대화 말뭉치를 읽어서 자연어 처리 및  Bag of word 생성
    """
    words = []
    classes = []
    documents = []
    ignore_words = ['?']
    # loop through each sentence in our intents patterns
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            # tokenize each word in the sentence
#             w = nltk.word_tokenize(pattern)
            pos_result = twitter.pos(pattern, norm=True, stem=True)
            w = [lex for lex, pos in pos_result]
            # add to our words list
            words.extend(w)
            # add to documents in our corpus
            documents.append((w, intent['tag']))
            # add to our classes list
            if intent['tag'] not in classes:
                classes.append(intent['tag'])
    
    # stem and lower each word and remove duplicates
#     words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
#     words = sorted(list(set(words)))
    words = [w for w in words if w not in ignore_words]
    words = sorted(list(set(words)))
    
    # remove duplicates
    classes = sorted(list(set(classes)))
    
    return classes, documents, words



In [5]:
def prepare_machine_learning(classes, documents, words):
    """
    Bag of word를 딥러닝 알고리즘 활용을 위한 입력으로 변환
    """
    
    # create our training data
    training = []
    output_row = []
    # create an empty array for our output
    output_empty = [0] * len(classes)
    
    # training set, bag of words for each sentence
    for doc in documents:
        # initialize our bag of words
        bag = []
        # list of tokenized words for the pattern
        pattern_words = doc[0]
        # stem each word
#         pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
        # create our bag of words array
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)
    
        # output is a '0' for each tag and '1' for current tag
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
    
        training.append([bag, output_row])
    
    # shuffle our features and turn into np.array
    random.shuffle(training)
    training = np.array(training)
    
    # create train and test lists
    train_x = list(training[:,0])
    train_y = list(training[:,1])

    return train_x, train_y



In [6]:
def create_tensorflow_learning_model(train_x, train_y, output_model_file_name):
    """
    딥러닝(tensorflow)을 통한 자연어 이해 모델 생성
    """
    
#     # reset underlying graph data
#     #tf.compat.v1.reset_default_graph()
#     tf.reset_default_graph()
#     # Build neural network
#     net = tflearn.input_data(shape=[None, len(train_x[0])])
#     net = tflearn.fully_connected(net, 8)
#     net = tflearn.fully_connected(net, 8)
#     net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
#     net = tflearn.regression(net)
    
#     # Define model and setup tensorboard
#     model = tflearn.DNN(net, tensorboard_dir='home_tflearn_kr_logs')
#     # Start training (apply gradient descent algorithm)
#     model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

    # Using Keras, Build neural network
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8, input_shape=(len(train_x[0]),)),
        tf.keras.layers.Dense(8),
        tf.keras.layers.Dense(len(train_y[0]), activation="softmax"),
        ])
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(train_x, train_y, epochs=1000, batch_size=8)    

    # save the trained model to directory
    model.save(output_model_file_name)

    return model



In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern
#     sentence_words = nltk.word_tokenize(sentence)
    pos_result = twitter.pos(sentence, norm=True, stem=True)
    sentence_words = [lex for lex, pos in pos_result]
    # stem each word
#     sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words



In [8]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

#     return(np.array(bag))
    return bag


In [9]:
# save all of our data structures
def save_training_data_structures(words, classes, train_x, train_y, output_training_data_file_name):
    """
    자연어 이해 모델을 관리한다(저장,읽기)
    """
    # save all of our data structures
    pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( output_training_data_file_name, "wb" ) )



In [10]:
# 대화 말뭉치 파일을 읽어들인다.
input_file_name = './DialogIntents/intents_home_kr.json'
intents = read_dialog_intents_jsonfile(input_file_name)
    
# 대화 말뭉치를 읽어서 자연어 처리 및  Bag of word 생성
classes, documents, words = dialog_nlp_processing(intents)
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
    


54 documents
11 classes ['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
77 unique stemmed words ['.', '가', '감사', '감사하다', '개발', '개선', '거기', '경력', '경험', '계세', '계시다', '고맙다', '과', '그렇다', '꺼지다', '나중', '날', '누구', '는', '다음', '닥치다', '돈', '들리다', '또', '를', '많이', '멍청이', '무슨', '바보', '버세', '번', '보완', '부분', '부족', '부탁', '사라지다', '사항', '새끼', '서비스', '소개', '수고', '안녕', '안녕하다', '야', '약점', '어떻다', '어학', '언제', '업적', '에', '에요', '여기', '영어', '요', '은', '을', '이', '이다', '일본어', '임마', '있다', '자기', '자다', '자신', '장사', '장점', '전공', '제품', '졸업', '좋다', '줄다', '중국어', '지내다', '친절하다', '하다', '학교', '해주다']


In [11]:
# Bag of word를 딥러닝 알고리즘 활용을 위한 입력으로 변환
train_x, train_y = prepare_machine_learning(classes, documents, words)

In [12]:
print(train_x[0])
print(len(train_x[0]))
print(train_y[0])
print(len(train_y[0]))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
77
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11


In [13]:
# 딥러닝(tensorflow)을 통한 자연어 이해 모델 생성
output_model_file_name = './NLUModel/model_home_kr_tf2'
model = create_tensorflow_learning_model(train_x, train_y, output_model_file_name)


Epoch 1/1000
7/7 [==============================] - 0s 1ms/step - loss: 2.4789 - accuracy: 0.0370
Epoch 2/1000
7/7 [==============================] - 0s 5ms/step - loss: 2.4525 - accuracy: 0.0370
Epoch 3/1000
7/7 [==============================] - 0s 3ms/step - loss: 2.4298 - accuracy: 0.1111
Epoch 4/1000
7/7 [==============================] - 0s 6ms/step - loss: 2.4064 - accuracy: 0.1296
Epoch 5/1000
7/7 [==============================] - 0s 1ms/step - loss: 2.3854 - accuracy: 0.1667
Epoch 6/1000
7/7 [==============================] - 0s 4ms/step - loss: 2.3638 - accuracy: 0.1667
Epoch 7/1000
7/7 [==============================] - 0s 5ms/step - loss: 2.3423 - accuracy: 0.1852
Epoch 8/1000
7/7 [==============================] - ETA: 0s - loss: 2.3464 - accuracy: 0.12 - 0s 1ms/step - loss: 2.3215 - accuracy: 0.1852
Epoch 9/1000
7/7 [==============================] - 0s 2ms/step - loss: 2.3011 - accuracy: 0.1667
Epoch 10/1000
7/7 [==============================] - 0s 6ms/step - loss: 2.2

7/7 [==============================] - 0s 2ms/step - loss: 0.6754 - accuracy: 0.8519
Epoch 84/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.6619 - accuracy: 0.8704
Epoch 85/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.6490 - accuracy: 0.8889
Epoch 86/1000
7/7 [==============================] - 0s 4ms/step - loss: 0.6363 - accuracy: 0.8889
Epoch 87/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.6238 - accuracy: 0.8889
Epoch 88/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.6113 - accuracy: 0.8889
Epoch 89/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.5992 - accuracy: 0.9074
Epoch 90/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.5878 - accuracy: 0.9074
Epoch 91/1000
7/7 [==============================] - 0s 4ms/step - loss: 0.5760 - accuracy: 0.9074
Epoch 92/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.5644 - accuracy: 0.9074
Epoch 93/1000
7/7 [=====

7/7 [==============================] - 0s 3ms/step - loss: 0.1643 - accuracy: 0.9630
Epoch 165/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.1622 - accuracy: 0.9630
Epoch 166/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.1611 - accuracy: 0.9444
Epoch 167/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.1585 - accuracy: 0.9444
Epoch 168/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.1567 - accuracy: 0.9630
Epoch 169/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.1545 - accuracy: 0.9630
Epoch 170/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.1537 - accuracy: 0.9630
Epoch 171/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.1515 - accuracy: 0.9630
Epoch 172/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.1496 - accuracy: 0.9630
Epoch 173/1000
7/7 [==============================] - 0s 5ms/step - loss: 0.1483 - accuracy: 0.9630
Epoch 174/1000


7/7 [==============================] - 0s 2ms/step - loss: 0.0862 - accuracy: 0.9630
Epoch 246/1000
7/7 [==============================] - 0s 998us/step - loss: 0.0859 - accuracy: 0.9630
Epoch 247/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.0857 - accuracy: 0.9630
Epoch 248/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0861 - accuracy: 0.9444
Epoch 249/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0851 - accuracy: 0.9444
Epoch 250/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0836 - accuracy: 0.9630
Epoch 251/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0839 - accuracy: 0.9630
Epoch 252/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0837 - accuracy: 0.9630
Epoch 253/1000
7/7 [==============================] - 0s 998us/step - loss: 0.0828 - accuracy: 0.9630
Epoch 254/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0832 - accuracy: 0.9630
Epoch 255/1

7/7 [==============================] - 0s 997us/step - loss: 0.0677 - accuracy: 0.9444
Epoch 327/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0672 - accuracy: 0.9630
Epoch 328/1000
7/7 [==============================] - 0s 998us/step - loss: 0.0677 - accuracy: 0.9444
Epoch 329/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0666 - accuracy: 0.9444
Epoch 330/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0670 - accuracy: 0.9444
Epoch 331/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0662 - accuracy: 0.9815
Epoch 332/1000
7/7 [==============================] - 0s 995us/step - loss: 0.0664 - accuracy: 0.9630
Epoch 333/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0667 - accuracy: 0.9630
Epoch 334/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0652 - accuracy: 0.9630
Epoch 335/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0654 - accuracy: 0.9630
Epo

...

7/7 [==============================] - 0s 997us/step - loss: 0.0540 - accuracy: 0.9630
Epoch 650/1000
7/7 [==============================] - 0s 998us/step - loss: 0.0549 - accuracy: 0.9630
Epoch 651/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0543 - accuracy: 0.9630
Epoch 652/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0539 - accuracy: 0.9630
Epoch 653/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0543 - accuracy: 0.9630
Epoch 654/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0540 - accuracy: 0.9630
Epoch 655/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0539 - accuracy: 0.9630
Epoch 656/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0541 - accuracy: 0.9630
Epoch 657/1000
7/7 [==============================] - 0s 4ms/step - loss: 0.0547 - accuracy: 0.9444
Epoch 658/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0550 - accuracy: 0.9444
Epo

7/7 [==============================] - 0s 1ms/step - loss: 0.0533 - accuracy: 0.9630
Epoch 731/1000
7/7 [==============================] - 0s 4ms/step - loss: 0.0537 - accuracy: 0.9630
Epoch 732/1000
7/7 [==============================] - 0s 855us/step - loss: 0.0539 - accuracy: 0.9630
Epoch 733/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0536 - accuracy: 0.9259
Epoch 734/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0541 - accuracy: 0.9444
Epoch 735/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0539 - accuracy: 0.9444
Epoch 736/1000
7/7 [==============================] - 0s 996us/step - loss: 0.0532 - accuracy: 0.9630
Epoch 737/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0534 - accuracy: 0.9630
Epoch 738/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0533 - accuracy: 0.9630
Epoch 739/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0535 - accuracy: 0.9630
Epoch

7/7 [==============================] - 0s 2ms/step - loss: 0.0534 - accuracy: 0.9444
Epoch 812/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0536 - accuracy: 0.9630
Epoch 813/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0535 - accuracy: 0.9444
Epoch 814/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.0534 - accuracy: 0.9630
Epoch 815/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0535 - accuracy: 0.9444
Epoch 816/1000
7/7 [==============================] - 0s 854us/step - loss: 0.0544 - accuracy: 0.9444
Epoch 817/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0532 - accuracy: 0.9630
Epoch 818/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0531 - accuracy: 0.9444
Epoch 819/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0532 - accuracy: 0.9444
Epoch 820/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0529 - accuracy: 0.9630
Epoch

Epoch 892/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0528 - accuracy: 0.9630
Epoch 893/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.0543 - accuracy: 0.9444
Epoch 894/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0529 - accuracy: 0.9630
Epoch 895/1000
7/7 [==============================] - 0s 855us/step - loss: 0.0530 - accuracy: 0.9444
Epoch 896/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0535 - accuracy: 0.9444
Epoch 897/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0525 - accuracy: 0.9630
Epoch 898/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0559 - accuracy: 0.9259
Epoch 899/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.0526 - accuracy: 0.9630
Epoch 900/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0524 - accuracy: 0.9630
Epoch 901/1000
7/7 [==============================] - 0s 997us/step - loss: 0.0532 - accuracy:

7/7 [==============================] - 0s 1ms/step - loss: 0.0526 - accuracy: 0.9630
Epoch 974/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.0546 - accuracy: 0.9630
Epoch 975/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0524 - accuracy: 0.9630
Epoch 976/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.0533 - accuracy: 0.9444
Epoch 977/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0528 - accuracy: 0.9630
Epoch 978/1000
7/7 [==============================] - 0s 5ms/step - loss: 0.0535 - accuracy: 0.9444
Epoch 979/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0528 - accuracy: 0.9444
Epoch 980/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0529 - accuracy: 0.9444
Epoch 981/1000
7/7 [==============================] - 0s 2ms/step - loss: 0.0525 - accuracy: 0.9630
Epoch 982/1000
7/7 [==============================] - 0s 3ms/step - loss: 0.0536 - accuracy: 0.9444
Epoch 983/1000


In [14]:
# 자연어 이해 모델을 관리한다(저장,읽기)
output_training_data_file_name = "./NLUModel/training_data_home_kr_tf2"
save_training_data_structures(words, classes, train_x, train_y, output_training_data_file_name)


In [ ]:
# train_x

In [ ]:
# [train_x[0]]

In [ ]:
# model.predict([train_x[0]])

In [ ]:
# model.predict_classes([train_x[0]])

In [ ]:
# predict_labels = np.argmax(model.predict([train_x[0]]), axis=1)
# predict_labels

In [ ]:
# [train_x[0]]

In [15]:
p = bow("자기자신 소개를 해주세요?", words)
# p

In [16]:
model.predict([p])

array([[1.1559612e-05, 5.3075250e-15, 4.4925014e-12, 4.5404569e-05,
        5.6648658e-10, 1.1828448e-07, 4.3433272e-07, 2.5220133e-15,
        9.9992478e-01, 3.2742402e-15, 1.7682067e-05]], dtype=float32)

In [17]:
model.predict_classes([p])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([8], dtype=int64)

In [18]:
print("p is Bag of word for '자기자신 소개를 해주세요??' :{}".format(p))
print("classes :{}".format(classes))
    
print("model.predict([p]) :{}".format(model.predict_classes([p])))
    
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

p is Bag of word for '자기자신 소개를 해주세요??' :[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
classes :['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
model.predict([p]) :[8]
54 documents
11 classes ['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
77 unique stemmed words ['.', '가', '감사', '감사하다', '개발', '개선', '거기', '경력', '경험', '계세', '계시다', '고맙다', '과', '그렇다', '꺼지다', '나중', '날', '누구', '는', '다음', '닥치다', '돈', '들리다', '또', '를', '많이', '멍청이', '무슨', '바보', '버세', '번', '보완', '부분', '부족', '부탁', '사라지다', '사항', '새끼', '서비스', '소개', '수고', '안녕', '안녕하다', '야', '약점', '어떻다', '어학', '언제', '업적', '에', '에요', '여기', '영어', '요', '은', '을', '이', '이다', '일본어', '임마', '있다', '자기', '자다', '자신', '장사', '장점', '전공', '제품', '졸업', '좋다', '줄다', '중국어', '지내다', '친절하다', '하다', '학교', '해주다']


In [19]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model(output_model_file_name)

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 624       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 11)                99        
Total params: 795
Trainable params: 795
Non-trainable params: 0
_________________________________________________________________


In [20]:
print("p is Bag of word for '자기자신 소개를 해주세요??' :{}".format(p))
print("classes :{}".format(classes))
    
print("model.predict([p]) :{}".format(new_model.predict_classes([p])))

p is Bag of word for '자기자신 소개를 해주세요??' :[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
classes :['Slang', 'goodbye', 'greeting', 'thanks', '경력', '약점', '어학', '자격', '자기소개', '장점', '학력']
model.predict([p]) :[8]
